In [1]:
import pandas as pd
import numpy as np
import os
from nbconvert import HTMLExporter
import nbformat
import matplotlib.pyplot as plt
import calendar
from datetime import datetime, timedelta
import re
import warnings

In [26]:
# Suprimir FutureWarnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [3]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows' , None)

nueva_ruta = r"\\ESSLIMD02610088\Users\edgar.coila\Desktop\Reportes Dra. Palomino\COMPILADO DE ALGORITMOS\Resultados Finales"
os.chdir(nueva_ruta)

In [4]:
# IMPORTANDO LA TABLA DE ATENCIONES 2022 Y 2023
ruta = r'atenciones2022_2023_id_encryt.csv'
Atenciones = pd.read_csv(ruta , sep = '|' , encoding='ISO-8859-1' )

C:\Users\ietsi.sdis2\AppData\Local\Temp\ipykernel_21288\3016630268.py:3: DtypeWarning: Columns (10,11) have mixed types. Specify dtype option on import or set low_memory=False.
  Atenciones = pd.read_csv(ruta , sep = '|' , encoding='ISO-8859-1' )


In [5]:
Atenciones = Atenciones.applymap(lambda x: x.strip() if isinstance(x, str) else x)


In [7]:
# Existen 'DIAGNOSTICO' con datos vacios, se van a retirar
Atenciones_1 = Atenciones.dropna(subset=['DIAGNOSTICO'])
# eliminando datos con sexo = 2
Atenciones_1 = Atenciones_1[Atenciones_1['categoria'] !="otros"]
Atenciones_1 = Atenciones_1[Atenciones_1['sexo'] !=2]
Atenciones_1['FECHA_ATENCION'] = pd.to_datetime(Atenciones_1['FECHA_ATENCION'], format='%d/%m/%Y')

In [8]:
# Se va a retirar de la base de datos Servicios: MEDICO PERSONAL,MEDICO CONTROL , MEDICO OCUPACIONAL Y DEL MEDIO AMBIENTE
Atenciones_1['SERVICIO'] = Atenciones_1['SERVICIO'].str.strip()

subcadenas = [
    'MEDICO DE PERSONAL',
    'MEDICINA OCUPACIONAL Y DEL MEDIO AMBIENTE',
    'MEDICO DE CONTROL \(MECO\)'
]

pattern = '|'.join(subcadenas)

Atenciones_1 = Atenciones_1[~Atenciones_1['SERVICIO'].str.contains(pattern, case=False, na=False)]

<>:7: SyntaxWarning: invalid escape sequence '\('
<>:7: SyntaxWarning: invalid escape sequence '\('
C:\Users\ietsi.sdis2\AppData\Local\Temp\ipykernel_21288\2989897706.py:7: SyntaxWarning: invalid escape sequence '\('
  'MEDICO DE CONTROL \(MECO\)'


In [9]:
# Diccionario de reemplazos
replacements = {
    'GINECOLOGIA': 'GINECOLOGIA Y OBSTETRICIA',
    'HEMATOLOGIA': 'HEMATOLOGIA CLINICA',
    'REANIMACION Y TERAPIA DEL DOLOR': 'ANESTESIA, ANALGESIA Y REANIMACION',
    'CIRUGIA VASCULAR': 'CIRUGIA DE TORAX Y CARDIOVASCULAR'
}

# Reemplazar los valores en la columna 'SERVICIO'
Atenciones_1['SERVICIO'] = Atenciones_1['SERVICIO'].replace(replacements)

In [ ]:
Atenciones_1.SERVICIO.unique()

In [11]:
Prime_Atenciones = Atenciones_1[['id','SERVICIO','DIAGNOSTICO','FECHA_ATENCION','categoria','anio','CENTRO']]

In [12]:
# Filtrando la base historica por grupo etario

Prime_Atenciones_A = Prime_Atenciones[Prime_Atenciones['categoria'] == '1m a < 5a']
Prime_Atenciones_B = Prime_Atenciones[Prime_Atenciones['categoria'] == '5a a < 18a']
Prime_Atenciones_C = Prime_Atenciones[Prime_Atenciones['categoria'] == 'de 18a a < de 30a']
Prime_Atenciones_D = Prime_Atenciones[Prime_Atenciones['categoria'] == 'de 30a a < de 60a']
Prime_Atenciones_E = Prime_Atenciones[Prime_Atenciones['categoria'] == 'de 60a a mÃ¡s']

In [13]:
# Importando los resultados de los votos
nueva_ruta = r"\\ESSLIMD02610088\Users\edgar.coila\Desktop\Reportes Dra. Palomino\COMPILADO DE ALGORITMOS"
os.chdir(nueva_ruta)

In [14]:
### VOTOS CON FILOSOFIA CALIDAD

# Grupo A
ruta = r'DF_CONSOLIDADO_A_respuesta_votos_C.txt'
datos_resultados_A_F_C = pd.read_csv(ruta , sep = '|' , encoding='ISO-8859-1' )
datos_resultados_A_F_C = datos_resultados_A_F_C.applymap(lambda x: x.strip() if isinstance(x, str) else x)
datos_resultados_A_F_C['_k'] = datos_resultados_A_F_C['_k'].astype(int)

# Grupo B
ruta = r'DF_CONSOLIDADO_B_respuesta_votos_C.txt'
datos_resultados_B_F_C = pd.read_csv(ruta , sep = '|' , encoding='ISO-8859-1' )
datos_resultados_B_F_C = datos_resultados_B_F_C.applymap(lambda x: x.strip() if isinstance(x, str) else x)
datos_resultados_B_F_C['_k'] = datos_resultados_B_F_C['_k'].astype(int)

# Grupo C
ruta = r'DF_CONSOLIDADO_C_respuesta_votos_C.txt'
datos_resultados_C_F_C = pd.read_csv(ruta , sep = '|' , encoding='ISO-8859-1' )
datos_resultados_C_F_C = datos_resultados_C_F_C.applymap(lambda x: x.strip() if isinstance(x, str) else x)
datos_resultados_C_F_C['_k'] = datos_resultados_C_F_C['_k'].astype(int)

# Grupo D
ruta = r'DF_CONSOLIDADO_D_respuesta_votos_C.txt'
datos_resultados_D_F_C = pd.read_csv(ruta , sep = '|' , encoding='ISO-8859-1' )
datos_resultados_D_F_C = datos_resultados_D_F_C.applymap(lambda x: x.strip() if isinstance(x, str) else x)
datos_resultados_D_F_C['_k'] = datos_resultados_D_F_C['_k'].astype(int)

# Grupo E
ruta = r'DF_CONSOLIDADO_E_respuesta_votos_C.txt'
datos_resultados_E_F_C = pd.read_csv(ruta , sep = '|' , encoding='ISO-8859-1' )
datos_resultados_E_F_C = datos_resultados_E_F_C.applymap(lambda x: x.strip() if isinstance(x, str) else x)
datos_resultados_E_F_C['_k'] = datos_resultados_E_F_C['_k'].astype(int)

### VOTOS CON FILOSOFIA AHORRO
# Grupo A
ruta = r'DF_CONSOLIDADO_A_respuesta_votos_A.txt'
datos_resultados_A_F_A = pd.read_csv(ruta , sep = '|' , encoding='ISO-8859-1' )
datos_resultados_A_F_A = datos_resultados_A_F_A.applymap(lambda x: x.strip() if isinstance(x, str) else x)
datos_resultados_A_F_A['_k'] = datos_resultados_A_F_A['_k'].astype(int)

# Grupo B
ruta = r'DF_CONSOLIDADO_B_respuesta_votos_A.txt'
datos_resultados_B_F_A = pd.read_csv(ruta , sep = '|' , encoding='ISO-8859-1' )
datos_resultados_B_F_A = datos_resultados_B_F_A.applymap(lambda x: x.strip() if isinstance(x, str) else x)
datos_resultados_B_F_A['_k'] = datos_resultados_B_F_A['_k'].astype(int)

# Grupo C
ruta = r'DF_CONSOLIDADO_C_respuesta_votos_A.txt'
datos_resultados_C_F_A = pd.read_csv(ruta , sep = '|' , encoding='ISO-8859-1' )
datos_resultados_C_F_A = datos_resultados_C_F_A.applymap(lambda x: x.strip() if isinstance(x, str) else x)
datos_resultados_C_F_A['_k'] = datos_resultados_C_F_A['_k'].astype(int)

# Grupo D
ruta = r'DF_CONSOLIDADO_D_respuesta_votos_A.txt'
datos_resultados_D_F_A = pd.read_csv(ruta , sep = '|' , encoding='ISO-8859-1' )
datos_resultados_D_F_A = datos_resultados_D_F_A.applymap(lambda x: x.strip() if isinstance(x, str) else x)
datos_resultados_D_F_A['_k'] = datos_resultados_D_F_A['_k'].astype(int)

# Grupo E
ruta = r'DF_CONSOLIDADO_E_respuesta_votos_A.txt'
datos_resultados_E_F_A = pd.read_csv(ruta , sep = '|' , encoding='ISO-8859-1' )
datos_resultados_E_F_A = datos_resultados_E_F_A.applymap(lambda x: x.strip() if isinstance(x, str) else x)
datos_resultados_E_F_A['_k'] = datos_resultados_E_F_A['_k'].astype(int)

In [27]:
def algortimo_asignacion_atenciones(atenciones: pd.DataFrame, resultados: pd.DataFrame, numero_digitos_cie_10: int) -> pd.DataFrame:
    warnings.simplefilter(action='ignore', category=FutureWarning)
    
    atenciones['RESULTADO DE TIPO ATENCION'] = 'SIN ASIGNACION'
    atenciones['REVISADO'] = 'N'
    total_iteraciones = len(resultados)

    for i, fila in resultados.iterrows():

        a = fila[0]  
        b = fila[1]  
        c = fila[2]  
        print(f"Analizando la iteración {i} de {total_iteraciones} , el diagnostico {a}")

        if numero_digitos_cie_10 == 3:
            a_regex = re.escape(a[:3]) 
            Prime_Atenciones_G = atenciones.loc[atenciones['DIAGNOSTICO'].str.match(a_regex) & (atenciones['REVISADO'] == 'N')]
            Prime_Atenciones_2023 = Prime_Atenciones_G.loc[Prime_Atenciones_G['anio'] == 2023]
        else:
            Prime_Atenciones_G = atenciones[(atenciones['DIAGNOSTICO'] == a) & (atenciones['REVISADO'] == 'N')]
            Prime_Atenciones_2023 = Prime_Atenciones_G.loc[Prime_Atenciones_G['anio'] == 2023]

        if b == 'MF' or b == 'MG':
            indices = Prime_Atenciones_G.index
            atenciones.loc[indices, 'RESULTADO DE TIPO ATENCION'] = b
            atenciones.loc[indices, 'REVISADO'] = 'S'
        
        if b != 'MF' and b != 'MG' :
        
            if not Prime_Atenciones_2023.empty:
                Atenciones_id = Prime_Atenciones_2023.sort_values(by='FECHA_ATENCION', ascending=True)
                for _, filax in Atenciones_id.iterrows():
                    e_id = filax['id']

                    primer_fecha_atencion_2023 = filax['FECHA_ATENCION']
                    f_un_anio_atras = primer_fecha_atencion_2023 - timedelta(days=365)

                    indices = Prime_Atenciones_G[
                                    (Prime_Atenciones_G['id'] == e_id) &
                                    (Prime_Atenciones_G['FECHA_ATENCION'] >= f_un_anio_atras) & (Prime_Atenciones_G['REVISADO'] == 'N')
                                ].index                     

                    if c == 1:
                            atenciones.loc[indices, ['RESULTADO DE TIPO ATENCION', 'REVISADO']] = [b, 'S']

                    if c == 2:
                            atenciones.loc[indices[:1], ['RESULTADO DE TIPO ATENCION', 'REVISADO']] = ['MF', 'S']
                            atenciones.loc[indices[1:], ['RESULTADO DE TIPO ATENCION', 'REVISADO']] = [b, 'S']

                    if c == 3:
                            atenciones.loc[indices[:2], ['RESULTADO DE TIPO ATENCION', 'REVISADO']] = ['MF', 'S']
                            atenciones.loc[indices[2:], ['RESULTADO DE TIPO ATENCION', 'REVISADO']] = [b, 'S']

                    if c == 4:
                            atenciones.loc[indices[:3], ['RESULTADO DE TIPO ATENCION', 'REVISADO']] = ['MF', 'S']
                            atenciones.loc[indices[3:], ['RESULTADO DE TIPO ATENCION', 'REVISADO']] = [b, 'S']

                    if c == 5:
                            atenciones.loc[indices[:4], ['RESULTADO DE TIPO ATENCION', 'REVISADO']] = ['MF', 'S']
                            atenciones.loc[indices[4:], ['RESULTADO DE TIPO ATENCION', 'REVISADO']] = [b, 'S']     

    warnings.simplefilter("default", FutureWarning)
    return atenciones

In [16]:
# Importando los resultados de los votos
nueva_ruta = r"\\ESSLIMD02610088\Users\edgar.coila\Desktop\Reportes Dra. Palomino\COMPILADO DE ALGORITMOS\Resultados Finales"
os.chdir(nueva_ruta)

In [ ]:
Resultado_A_A_3D_Nuevo = algortimo_asignacion_atenciones(Prime_Atenciones_A , datos_resultados_A_F_A,3)
Resultado_A_A_3D_Nuevo = Resultado_A_A_3D_Nuevo.sort_index()
Resultado_A_A_3D_Nuevo.to_csv('Resultado_A_A_3D_Nuevo.txt', index=False , sep = '|')

In [ ]:
Resultado_E_C_3D_Nuevo = algortimo_asignacion_atenciones(Prime_Atenciones_E , datos_resultados_E_F_C,3)
Resultado_E_C_3D_Nuevo = Resultado_E_C_3D_Nuevo.sort_index()
Resultado_E_C_3D_Nuevo.to_csv('Resultado_E_C_3D_Nuevo.txt', index=False , sep = '|')

In [ ]:
Resultado_D_C_3D_Nuevo = algortimo_asignacion_atenciones(Prime_Atenciones_D , datos_resultados_D_F_C,3)
Resultado_D_C_3D_Nuevo = Resultado_D_C_3D_Nuevo.sort_index()
Resultado_D_C_3D_Nuevo.to_csv('Resultado_D_C_3D_Nuevo.txt', index=False , sep = '|')


In [ ]:
Resultado_C_C_3D_Nuevo = algortimo_asignacion_atenciones(Prime_Atenciones_C , datos_resultados_C_F_C,3)
Resultado_C_C_3D_Nuevo = Resultado_C_C_3D_Nuevo.sort_index()
Resultado_C_C_3D_Nuevo.to_csv('Resultado_C_C_3D_Nuevo.txt', index=False , sep = '|')

In [ ]:
Resultado_B_C_3D_Nuevo = algortimo_asignacion_atenciones(Prime_Atenciones_B , datos_resultados_B_F_C,3)
Resultado_B_C_3D_Nuevo = Resultado_B_C_3D_Nuevo.sort_index()
Resultado_B_C_3D_Nuevo.to_csv('Resultado_B_C_3D_Nuevo.txt', index=False , sep = '|')

In [ ]:
Resultado_A_C_3D_Nuevo = algortimo_asignacion_atenciones(Prime_Atenciones_A , datos_resultados_A_F_C,3)
Resultado_A_C_3D_Nuevo = Resultado_A_C_3D_Nuevo.sort_index()
Resultado_A_C_3D_Nuevo.to_csv('Resultado_A_C_3D_Nuevo.txt', index=False , sep = '|')

In [ ]:
Resultado_E_A_3D_Nuevo = algortimo_asignacion_atenciones(Prime_Atenciones_E , datos_resultados_E_F_A,3)
Resultado_E_A_3D_Nuevo = Resultado_E_A_3D_Nuevo.sort_index()
Resultado_E_A_3D_Nuevo.to_csv('Resultado_E_A_3D_Nuevo.txt', index=False , sep = '|')

In [ ]:
Resultado_D_A_3D_Nuevo = algortimo_asignacion_atenciones(Prime_Atenciones_D , datos_resultados_D_F_A,3)
Resultado_D_A_3D_Nuevo = Resultado_D_A_3D_Nuevo.sort_index()
Resultado_D_A_3D_Nuevo.to_csv('Resultado_D_A_3D_Nuevo.txt', index=False , sep = '|')

In [ ]:
Resultado_E_C_4D_Nuevo = algortimo_asignacion_atenciones(Prime_Atenciones_E , datos_resultados_E_F_C,4)
Resultado_E_C_4D_Nuevo = Resultado_E_C_4D_Nuevo.sort_index()
Resultado_E_C_4D_Nuevo.to_csv('Resultado_E_C_4D_Nuevo.txt', index=False , sep = '|')